## Dipolar Wakefield simulation of the LHC unshielded bellows with `wakis`

* Creation of the geometry from simple geometry blocks (CSG) -> fully parametrized
* Simulation of the Dipolar X impedance and wake
  
| Wake | Impedance |
| --- | ---------- |
|     |            | 

In [22]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt

from wakis import SolverFIT3D
from wakis import GridFIT3D 
from wakis import WakeSolver

%matplotlib ipympl

### Geometry generation using Constructive Solid Geometry (CSG)

In [23]:
# beam pipe
r_pipe = 24e-3      # pipe radius [m]
l_pipe = 320e-3     # pipe length [m]
pipe = pv.Cylinder(center=(0,0,0), direction=(0,0,1), radius=r_pipe, height=l_pipe)

# convolutions
r_conv = 30e-3  # convolution radius [m]
l_conv = 4e-3  # length of each convolution [m]
n_conv = 15      # number of convolutions
l_between_conv = 4e-3  # length between convolutions [m]
z_start = n_conv//2*(l_conv + l_between_conv) - l_conv # start of the convolutions [m]
convolutions = []

for n in range(n_conv):
    z_start_n_conv = -z_start+n*(l_conv+l_between_conv)
    conv = pv.Cylinder(center=(0,0,z_start_n_conv),     # center of the convolution
                       direction=(0,0,1),  # z-direction
                       radius=r_conv, 
                       height=l_conv)
    convolutions.append(conv)  # append to list

# Sum the generated geometry
pipe = pipe.triangulate()                          # triangulate pipe
convolutions = np.sum(convolutions).triangulate()  # triangulate convolutions
bellow = pipe + convolutions  # union of meshes without internal faces

# Save it to file
stl_file = 'notebooks/data/006_LHC_Bellow_generated.stl'
bellow.save(stl_file) #save in [m]
# bellow.scale(1e3).save(stl_file) #save in [mm]

# Plot the generated geometry
bellow.clear_data()   
#bellow.plot(opacity=0.7)

### Domain setup and grid generation

In [ ]:
# ---------- Domain setup ---------
# Set up geometry & materials dictionaries
stl_solids = {'bellow': stl_file}
stl_materials = {'bellow': 'vacuum'}

# Domain bounds
xmin, xmax, ymin, ymax, zmin, zmax = bellow.bounds

# Number of mesh cells
Nx = 60
Ny = 60
Nz = 220
print(f'Total number of mesh cells: {Nx*Ny*Nz}')

# set grid and geometry
grid = GridFIT3D(xmin, xmax, ymin, ymax, zmin, zmax, Nx, Ny, Nz, 
                stl_solids=stl_solids, 
                stl_materials=stl_materials,
                use_mesh_refinement=True,
                refinement_tol=1e-8,)

Total number of mesh cells: 792000
Generating grid...
Applying mesh refinement...
* Calculating snappy points...
* Refining x axis with 5 snaps...
Both `ftol` and `xtol` termination conditions are satisfied.
Function evaluations 25, initial cost 3.8333e-04, final cost 1.8979e-05, first-order optimality 5.88e-04.
* Refining y axis with 5 snaps...
Both `ftol` and `xtol` termination conditions are satisfied.
Function evaluations 25, initial cost 3.8333e-04, final cost 1.8979e-05, first-order optimality 5.88e-04.
* Refining z axis with 32 snaps...


In [ ]:
grid.inspect()

### Boundary conditions and EM solver

In [ ]:
# boundary conditions
bc_low=['pec', 'pec', 'pml']
bc_high=['pec', 'pec', 'pml']

n_pml = 10 # number of PML layers
solver = SolverFIT3D(grid, 
                     bc_low=bc_low, bc_high=bc_high, 
                     use_stl=True, bg='pec',
                     n_pml=n_pml)

### Wakefield settings & Run

The wakefield simulation will run up to 10 m which will give a partially decayed wake and a non-converged impedance - this will be the starting point of IDDEFIX extrapolation !

In [ ]:
# ------------ Beam source ----------------
# Beam parameters
sigmaz = 10e-3      #[m] -> 2 GHz
q = 1e-9            #[C]
beta = 1.0          # beam beta 
xs = 5e-3           # x source position [m]
ys = 0.             # y source position [m]
xt = 0.             # x test position [m]
yt = 0.             # y test position [m]
# [DEFAULT] tinj = 8.53*sigmaz/c_light  # injection time offset [s] 

# ----------- Solver  setup  ----------
# Wakefield post-processor
wakelength = 10. # [m] -> Partially decayed
skip_cells = n_pml + 2  # no. cells to skip from wake integration (>= PML cells)

results_folder = 'notebooks/006_results/'
wake = WakeSolver(q=q, sigmaz=sigmaz, beta=beta,
                xsource=xs, ysource=ys, xtest=xt, ytest=yt,
                skip_cells=skip_cells, wakelength=wakelength,
                results_folder=results_folder,
                Ez_file=results_folder+'Ez.h5',)

In [ ]:
# Plot settings
import os
if not os.path.exists(results_folder+'img/'): os.mkdir(results_folder+'img/')
plotkw = {'title':results_folder+'img/Ez', 
            'add_patch':'bellow', 'patch_alpha':0.3,
            'vmin':-1e4, 'vmax':1e4,
            'plane': [int(Nx/2), slice(0, Ny), slice(0, Nz)],}

solver.wakesolve(wakelength=wakelength, 
                 wake=wake, 
                 plot=False,
                 plot_every=50, 
                 plot_until=5000,
                 **plotkw)

Or, if results have already been generated:

In [ ]:
wake.load_results(results_folder)

# Modify the number of cells skip from the boundary during integration
# to aboid boundary artifacts -useful in low impedance devices!
wake.skip_cells = 30
wake.solve()

It's done! Now we can plot the results:

In [ ]:
# Plot longitudinal wake potential and impedance
fig1, ax = plt.subplots(2,1, figsize=[8,8], dpi=150)
ax[0].plot(wake.s*1e2, wake.WP, c='tab:red', lw=1.5, label='Wakis')
ax[0].set_xlabel('s [cm]')
ax[0].set_ylabel('Longitudinal wake potential [V/pC]', color='tab:red')
ax[0].legend()
ax[0].set_xlim(xmax=wakelength*1e2)

ax[1].plot(wake.f*1e-9, np.abs(wake.Z), c='tab:blue', alpha=0.8, lw=2, label='Abs')
ax[1].plot(wake.f*1e-9, np.real(wake.Z), ls='--', c='tab:blue', lw=1.5, label='Real')
ax[1].plot(wake.f*1e-9, np.imag(wake.Z), ls=':', c='tab:blue', lw=1.5, label='Imag')
ax[1].set_xlabel('f [GHz]')
ax[1].set_ylabel('Longitudinal impedance [Abs][$\Omega$]', color='tab:blue')
ax[1].legend()

fig1.tight_layout()
#fig1.savefig(results_folder+'longitudinal.png')
#plt.show()

In [ ]:
# Plot transverse x wake potential and impedance
fig2, ax = plt.subplots(2,1, figsize=[8,8], dpi=150)
ax[0].plot(wake.s*1e2, wake.WPx, c='tab:orange', lw=1.5, label='Wakis')
ax[0].set_xlabel('s [cm]')
ax[0].set_ylabel('Transverse wake potential X [V/pC]', color='tab:orange')
ax[0].legend()
ax[0].set_xlim(xmax=wakelength*1e2)

ax[1].plot(wake.f*1e-9, np.abs(wake.Zx), c='tab:green', lw=2, label='Abs')
ax[1].plot(wake.f*1e-9, np.real(wake.Zx), c='tab:green', ls='--', lw=1.5, label='Real')
ax[1].plot(wake.f*1e-9, np.imag(wake.Zx), c='tab:green', ls=':', lw=1.5, label='Imag')
ax[1].set_xlabel('f [GHz]')
ax[1].set_ylabel('Transverse impedance X [Abs][$\Omega$]', color='tab:green')
ax[1].legend()

fig2.tight_layout()
#fig2.savefig(results_folder+'transverse_x.png')
#plt.show()

In [ ]:
# Plot transverse y wake potential and impedance
fig3, ax = plt.subplots(2,1, figsize=[8,8], dpi=150)
ax[0].plot(wake.s*1e2, wake.WPy, c='tab:brown', lw=1.5, label='Wakis')
ax[0].set_xlabel('s [cm]')
ax[0].set_ylabel('Transverse wake potential Y [V/pC]', color='tab:brown')
ax[0].legend()
ax[0].set_xlim(xmax=wakelength*1e2)

ax[1].plot(wake.f*1e-9, np.abs(wake.Zy), c='tab:pink', lw=2, label='Abs')
ax[1].plot(wake.f*1e-9, np.real(wake.Zy), c='tab:pink', ls='--', lw=1.5, label='Real')
ax[1].plot(wake.f*1e-9, np.imag(wake.Zy), c='tab:pink', ls=':', lw=1.5, label='Imag')
ax[1].set_xlabel('f [GHz]')
ax[1].set_ylabel('Transverse impedance Y [Abs][$\Omega$]', color='tab:pink')
ax[1].legend()

fig3.tight_layout()
#fig3.savefig(results_folder+'transverse_y.png')
#plt.show()

### Compare with CST results

In [ ]:
WP_cst = wake.read_txt('notebooks/data/006_CST_dipolar_x_wake.txt')
Zx_cst = wake.read_txt('notebooks/data/006_CST_dipolar_x_impedance.txt')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=[12,4], dpi=100)

ax[0].plot(WP_cst[0], WP_cst[1], c='k', lw=1.5, label='WPx CST')

ax[0].plot(wake.s*1e3, wake.WPx, c='darkgreen', lw=1.5, alpha=0.5, label='WPx Wakis')
ax[0].set_xlabel('s [mm]')
ax[0].set_ylabel('Transverse wake potential X [V/pC]', color='darkgreen')

ax[1].plot(Zx_cst[0], Zx_cst[1], c='k', lw=1.5, label='Re(Zx) CST')
ax[1].plot(Zx_cst[0], Zx_cst[2], c='k', ls='--', lw=1.5, label='Im(Zx) CST')

ax[1].plot(wake.f*1e-9, np.real(wake.Zx), c='g', alpha=0.5, lw=1.5, label='Re(Zx) Wakis')
ax[1].plot(wake.f*1e-9, -1*np.imag(wake.Zx), c='g', ls='--', alpha=0.5, lw=1.5, label='Im(Zx) Wakis')
ax[1].legend()
ax[1].set_xlabel('f [GHz]')
ax[1].set_ylabel('Transverse impedance X [Re/Im][$\Omega$]', color='g')

fig.tight_layout()